In [16]:
import GPy
# GPy.tests()
import numpy as np
import h5py
import scipy.stats
import sklearn.model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [6]:
with h5py.File('MOS_Scores.mat', 'r') as f:
	for k,v in f.items():
		Y = (np.array(v))
		Y = Y.reshape(Y.shape[1])
		np.save('Y_mat.npy',Y)
with h5py.File('konvid_features.mat', 'r') as f:
	for k,v in f.items():
		X = (np.array(v)).T
		np.save('X_mat.npy',X)

		print(X.shape)

		# X1 = (X**2).copy()
		# X = np.hstack((X,X1))
		
		X = StandardScaler().fit_transform(X)
        
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.2,shuffle=True,random_state=42)
x_train,x_val,y_train,y_val = x_train[:900].copy(), x_train[900:].copy(),y_train[:900].copy(),y_train[900:].copy()

y_train,y_val,y_test = np.expand_dims(y_train,axis=1),np.expand_dims(y_val,axis=1),np.expand_dims(y_test,axis=1)

print(x_train.shape,x_val.shape,x_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)

(1200, 100)
(900, 100) (60, 100) (240, 100)
(900, 1) (60, 1) (240, 1)


In [27]:
k1 = GPy.kern.RBF(input_dim=x_train.shape[1])
k2 = GPy.kern.Matern52(input_dim=x_train.shape[1])
kernel = GPy.kern. k1*k2
m = GPy.models.GPRegression(x_train,y_train,kernel)
print (m)


Name : GP regression
Objective : 3289.338131492887
Number of Parameters : 5
Number of Optimization Parameters : 5
Updates : True
Parameters:
  GP_regression.           |  value  |  constraints  |  priors
  mul.rbf.variance         |    1.0  |      +ve      |        
  mul.rbf.lengthscale      |    1.0  |      +ve      |        
  mul.Mat52.variance       |    1.0  |      +ve      |        
  mul.Mat52.lengthscale    |    1.0  |      +ve      |        
  Gaussian_noise.variance  |    1.0  |      +ve      |        


In [28]:
m.randomize()
m.optimize_restarts(num_restarts = 5,max_iters=5000)

 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:565: RuntimeWarning:invalid value encountered in multiply
 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply
 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:568: RuntimeWarning:invalid value encountered in subtract


Optimization restart 1/5, f = 646.0300802973072
Optimization restart 2/5, f = 2294.6193111208495


 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:565: RuntimeWarning:overflow encountered in square
 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:568: RuntimeWarning:overflow encountered in square
 /home/harsh/anaconda3/lib/python3.7/site-packages/GPy/kern/src/stationary.py:568: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 621.2326391180844
Optimization restart 4/5, f = 621.2265843692296
Optimization restart 5/5, f = 2294.6193111208527


In [29]:
print (m)


Name : GP regression
Objective : 621.2265843692296
Number of Parameters : 5
Number of Optimization Parameters : 5
Updates : True
Parameters:
  GP_regression.           |                 value  |  constraints  |  priors
  mul.rbf.variance         |     541.7618653709078  |      +ve      |        
  mul.rbf.lengthscale      |     5482.997257435856  |      +ve      |        
  mul.Mat52.variance       |  0.013420836239793038  |      +ve      |        
  mul.Mat52.lengthscale    |    37.465540864542625  |      +ve      |        
  Gaussian_noise.variance  |     0.188762458785246  |      +ve      |        


In [30]:
y_pred = m.predict(x_test)[0]
y_pred = y_pred.reshape(y_test.shape)

In [31]:
print("pearson",np.corrcoef(y_pred,y_test)[0,1])
print("Spearman",scipy.stats.spearmanr(y_pred, y_test, axis=0)[0])

pearson 0.6580302198611977
Spearman 0.6593202883649387


In [32]:
np.corrcoef(y_test,y_pred)

array([[1.        , 0.65803022],
       [0.65803022, 1.        ]])